\begin{equation}
\text{VaR}_{\alpha}(X) = \text{inf} \{ x \in \mathbb{R} \mid P(X \leq x) \geq 1 - \alpha \}
\end{equation}


$$
\text{VaR}_{\alpha}(X) = \mu_X + \sigma_X \cdot \Phi^{-1}(\alpha)
$$

where:
- $\text{VaR}_{\alpha}(X)$ represents the Value at Risk of the portfolio or position $X$ at the confidence level $\alpha$.
- $\mu_X$ represents the mean or expected return of the portfolio or position $X$.
- $\sigma_X$ represents the standard deviation of the portfolio or position $X$.
- $\Phi^{-1}(\alpha)$ represents the inverse cumulative distribution function (quantile function) of the standard normal distribution, evaluated at the confidence level $\alpha$.


In [6]:
import scipy.stats
import numpy as np

In [3]:
scipy.stats.norm.ppf(0.95)

1.6448536269514722

In [4]:
def VaR(Position,sigma,Quantile):
    return Position*sigma*scipy.stats.norm.ppf(Quantile)

In [5]:
VaR(1000,0.05,0.95)

82.24268134757361

$$
\sigma^2_p = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij}
$$

where:
- \(\sigma^2_p\) represents the portfolio variance.
- \(w_i\) and \(w_j\) represent the weights of assets \(i\) and \(j\) in the portfolio, respectively.
- \(\sigma_{ij}\) represents the covariance between assets \(i\) and \(j\).
- \(n\) represents the number of assets in the portfolio.



In [7]:
w1=0.5
w2=0.5
sigma1=0.07
sigma2=0.03
corr=0.4

In [8]:
portfoliovar=w1**2*sigma1**2+w2**2*sigma2**2+2*(w1*w2*sigma1*sigma2*corr)

In [9]:
portfoliovol=portfoliovar**(1/2)

In [10]:
portfoliovol

0.043243496620879306

In [11]:
VaR(1000,portfoliovol,0.95)

71.12922225891705

In [14]:
VaR1=VaR(1000*w1,sigma1,0.95)
VaR2=VaR(1000*w2,sigma2,0.95)

In [16]:
vector=np.array([VaR1,VaR2])
vector

array([57.56987694, 24.6728044 ])

In [17]:
corrmatrix=np.array([[1,corr],[corr,1]])
corrmatrix

array([[1. , 0.4],
       [0.4, 1. ]])

In [21]:
(np.dot(np.dot(vector,corrmatrix),vector))**(1/2)

71.12922225891707

In [22]:
import yfinance as yf
import datetime as dt

In [23]:
end=dt.datetime.now()
start=dt.datetime(end.year-1,end.month,end.day)

In [26]:
df=yf.download(['AAPL','MSFT','TSLA'],start,end)['Adj Close']
df

[*********************100%%**********************]  3 of 3 completed


,AAPL,MSFT,TSLA
Date,,,
2023-04-17,164.352737,286.449036,187.039993
2023-04-18,165.586151,286.022522,184.309998
2023-04-19,166.740005,286.101898,180.589996
2023-04-20,165.765182,283.780914,162.990005
2023-04-21,164.143860,283.433807,165.080002
...,...,...,...
2024-04-09,169.669998,426.279999,176.880005
2024-04-10,167.779999,423.260010,171.759995
2024-04-11,175.039993,427.929993,174.600006


In [29]:
returns=np.log(1+df.pct_change())
returns

,AAPL,MSFT,TSLA
Date,,,
2023-04-17,NaN,NaN,NaN
2023-04-18,0.007477,-0.001490,-0.014703
2023-04-19,0.006944,0.000277,-0.020390
2023-04-20,-0.005864,-0.008146,-0.102540
2023-04-21,-0.009829,-0.001224,0.012741
...,...,...,...
2024-04-09,0.007216,0.003972,0.022296
2024-04-10,-0.011202,-0.007110,-0.029373
2024-04-11,0.042361,0.010973,0.016400


In [30]:
returns.std()

AAPL    0.012314
MSFT    0.013772
TSLA    0.030425
dtype: float64

In [33]:
Position=df.iloc[-1]
Position

AAPL    173.830002
MSFT    414.720001
TSLA    162.860306
Name: 2024-04-15 00:00:00, dtype: float64

In [35]:
VaRarray=[]
for i in range(len(Position)):
    VaRarray.append(VaR(Position[i],returns.std()[i],0.95))

In [36]:
VaRarray

[3.5208764073460173, 9.394462650976914, 8.15025163932988]

In [37]:
vector=np.array(VaRarray)
vector

array([3.52087641, 9.39446265, 8.15025164])

In [38]:
returns.corr()

,AAPL,MSFT,TSLA
AAPL,1.000000,0.478074,0.356267
MSFT,0.478074,1.000000,0.264795
TSLA,0.356267,0.264795,1.000000


In [41]:
(np.dot(np.dot(vector,returns.corr()),vector))**(1/2)

16.115255658492973